In [1]:
from opsml_artifacts import DataCard, CardRegistry, DriftDetector, DriftVisualizer, LocalSQLConnection, CloudSQLConnection
from opsml_artifacts.connector import SnowflakeQueryRunner
import numpy as np

In [2]:
local_conn = LocalSQLConnection()
data_registry = CardRegistry(registry_name="data", connection_client=local_conn)

In [3]:
data_registry.list_cards()

,uid,date,timestamp,app_env,name,team,version,user_email,data_uri,drift_uri,feature_map,feature_descriptions,data_splits,data_type,additional_info,dependent_vars
0,aaf00ac219f6482386d61de10a8a89f2,2023-02-07,1675792654344,development,test_data,mlops,3,mlops.com,/home/steven.forrester/DATA_REGISTRY/mlops/tes...,None,{'numpy_dtype': 'int64'},None,None,ndarray,{},None
1,c8160a98540240dca617d2db2fe56048,2023-02-07,1675788694904,development,test_data,mlops,2,mlops.com,/home/steven.forrester/DATA_REGISTRY/mlops/tes...,None,{'numpy_dtype': 'int64'},None,None,ndarray,{},None
2,ec910d7fc2b744cc97cd14b37a31211e,2023-02-07,1675806458392,development,test_data1,mlops,2,mlops.com,/home/steven.forrester/DATA_REGISTRY/mlops/tes...,None,{'numpy_dtype': 'int64'},None,None,ndarray,{},None
3,3eac180bd3704ed3944fcca25fb2ac56,2023-02-07,1675806458392,development,test_data2,mlops,2,mlops.com,/home/steven.forrester/DATA_REGISTRY/mlops/tes...,None,{'numpy_dtype': 'int64'},None,None,ndarray,{},None
4,56f8058f530a480e93b5823eb653b8a9,2023-02-07,1675788654445,development,test_data,mlops,1,mlops.com,/home/steven.forrester/DATA_REGISTRY/mlops/tes...,None,{'numpy_dtype': 'int64'},None,None,ndarray,{},None
5,45caf5b35edc49bfa14111be623172da,2023-02-07,1675801823773,development,test_data1,mlops,1,mlops.com,/home/steven.forrester/DATA_REGISTRY/mlops/tes...,None,{'numpy_dtype': 'int64'},None,None,ndarray,{},None
6,0beb07f18da04d81b8d5545783f20a1b,2023-02-07,1675801823773,development,test_data2,mlops,1,mlops.com,/home/steven.forrester/DATA_REGISTRY/mlops/tes...,None,{'numpy_dtype': 'int64'},None,None,ndarray,{},None


In [4]:
query_runner = SnowflakeQueryRunner(on_vpn=True) 
#query runner is a temporary wrapper for pyshipt sql (needed for network issues in vertex, see opsml-pipelines docs)

In [5]:
dataframe = query_runner.query_to_dataframe(sql_file="data.sql") 
#executes sql file or raw sql. data.sql is in examples dir

In [6]:
dataframe.head()

,TIME_BUNDLE_ID,LOCAL_CREATED_DATE,LOCAL_CREATED_AT,METRO_ID,BUNDLE_TYPE,NBR_ADDRESSES,NBR_ORDERS,DROP_OFF_TIME,EVAL_FLG,EVAL_OUTLIER,...,COS_DAY_YR,SIN_MTH,COS_MTH,SIN_WK_YR,COS_WK_YR,NG_ORDER_ID,METRO_X,METRO_Y,METRO_Z,APT_FLG
0,053c3646-03a1-4528-bb25-5c3f072cc7d9,2022-06-03,2022-06-03 18:22:25.231270,13,TARP,1,1,4.394073,0,0,...,-0.882049,1.224647e-16,-1.000000,0.464723,-0.885456,E_132853608,824.546550,-5135.838323,3678.712915,0
1,c7f0d060-44d4-4f41-9210-8879b0469655,2022-06-05,2022-06-05 18:36:36.475652,137,TARP,2,2,9.754285,0,0,...,-0.897743,1.224647e-16,-1.000000,0.464723,-0.885456,None,-2498.363640,-3835.840543,4431.043607,0
2,71b6e060-7950-4d4d-bba1-6f09e033beb5,2022-05-31,2022-05-31 20:13:30.499271,48,TARP,6,6,20.253595,0,0,...,-0.856552,4.999982e-01,-0.866026,0.464723,-0.885456,None,-676.719393,-5314.674399,3447.597409,1
3,2d04f063-13a9-430c-a82f-1f1a4040c46c,2022-06-02,2022-06-02 09:36:24.218670,278,TARP,1,2,8.030227,0,0,...,-0.873807,1.224647e-16,-1.000000,0.464723,-0.885456,None,-2499.949432,-4649.471724,3566.834244,0
4,24f2a1a3-6326-4c76-8ac5-8df11c94ddf5,2022-06-04,2022-06-04 15:43:42.629295,279,TARP,2,2,11.791392,0,0,...,-0.890029,1.224647e-16,-1.000000,0.464723,-0.885456,None,-2476.129544,-4684.469262,3537.537451,1


In [8]:
features = [
    "NBR_ADDRESSES",
    "NBR_ORDERS",
    "NBR_RX",
    "NBR_APT",
    "METRO_X",
    "METRO_Y",
    "METRO_Z",
    "APT_FLG",
    "DROP_OFF_TIME",
    "EVAL_FLG",
]
DEPENDENT_VAR = "DROP_OFF_TIME"

# Define DataCard attributes (see examples dir for more detailed information)
DATA_NAME = "tarp_drop_off"
TEAM = "SPMS"
USER_EMAIL = "steven.forrester@shipt.com"
DATA_SPLITS = [
    {"label": "train", "column": "EVAL_FLG", "column_value": 0},
    {"label": "test", "column": "EVAL_FLG", "column_value": 1},
]

data_card = DataCard(
    data=dataframe[features],
    name=DATA_NAME,
    team=TEAM,
    user_email=USER_EMAIL,
    data_splits=DATA_SPLITS,
    dependent_vars=[DEPENDENT_VAR],

)

data_registry = CardRegistry(registry_name="data", connection_client=local_conn)
data_registry.register_card(card=data_card)

{"message": "DATA_REGISTRY: tarp_drop_off registered as version 2", "timestamp": "2023-02-09T00:57:22.131787Z", "app_env": "development", "level": "INFO"}


In [1]:
from opsml_artifacts import CardRegistry, LocalSQLConnection

local_conn = LocalSQLConnection()
data_registry = CardRegistry(registry_name="data", connection_client=local_conn)
tarp_list = data_registry.list_cards(team="SPMS", name="tarp_drop_off")

In [2]:
print(tarp_list.loc[:, ~tarp_list.columns.isin(["feature_map", "data_splits", "drift_uri"])].to_markdown())

|    | uid                              | date       |     timestamp | app_env     | name          | team   |   version | user_email                 | data_uri                                                                                                   | feature_descriptions   | data_type   | additional_info   | dependent_vars    |
|---:|:---------------------------------|:-----------|--------------:|:------------|:--------------|:-------|----------:|:---------------------------|:-----------------------------------------------------------------------------------------------------------|:-----------------------|:------------|:------------------|:------------------|
|  0 | e18e5bd3fbb145a182d0fe61e24b1d66 | 2023-02-09 | 1675904151384 | development | tarp_drop_off | SPMS   |         2 | steven.forrester@shipt.com | /home/steven.forrester/DATA_REGISTRY/SPMS/tarp_drop_off/version-2/6de7a703618e4718a39f301b50dda8dc.parquet |                        | DataFrame   | {}                | ['D

In [4]:
loaded_card = data_registry.load_card(uid="e18e5bd3fbb145a182d0fe61e24b1d66") 
loaded_card.load_data() # data is not automatically loaded with the card (prevents loading issues with big data)
# load_card can take a few arguments. Be sure to check to docstring

In [5]:
print(loaded_card.data.head().to_markdown())

|    |   NBR_ADDRESSES |   NBR_ORDERS |   NBR_RX |   NBR_APT |   METRO_X |   METRO_Y |   METRO_Z |   APT_FLG |   DROP_OFF_TIME |   EVAL_FLG |
|---:|----------------:|-------------:|---------:|----------:|----------:|----------:|----------:|----------:|----------------:|-----------:|
|  0 |               1 |            1 |        0 |         0 |   824.547 |  -5135.84 |   3678.71 |         0 |         4.39407 |          0 |
|  1 |               2 |            2 |        0 |         0 | -2498.36  |  -3835.84 |   4431.04 |         0 |         9.75429 |          0 |
|  2 |               6 |            6 |        0 |         1 |  -676.719 |  -5314.67 |   3447.6  |         1 |        20.2536  |          0 |
|  3 |               1 |            2 |        0 |         0 | -2499.95  |  -4649.47 |   3566.83 |         0 |         8.03023 |          0 |
|  4 |               2 |            2 |        0 |         2 | -2476.13  |  -4684.47 |   3537.54 |         1 |        11.7914  |          0 |


In [10]:
from opsml_artifacts import ModelCard, DriftDetector
from lightgbm import LGBMRegressor

data_splits = loaded_card.split_data()

In [11]:
data_splits = loaded_card.split_data() # get the data splits defined by split logic (data_card.data_splits)

# Prepare train data
data_splits.train.pop("EVAL_FLG") # pop off eval flg
y_train = data_splits.train.pop("DROP_OFF_TIME") # get train target

# Prepare test data
data_splits.test.pop("EVAL_FLG") # pop off eval flg
y_test = data_splits.test.pop("DROP_OFF_TIME") # get train target

#### Drift report

In [12]:
drift_detector = DriftDetector(x_reference=data_splits.train,
                              x_current=data_splits.test,
                              y_reference=y_train.to_numpy().reshape(-1,1),
                              y_current=y_test.to_numpy().reshape(-1,1),
                              dependent_var_name="DROP_OFF_TIME",
                             categorical_features=["NBR_ORDERS"],
                             )
drift_detector.visualize_report()

alt.VConcatChart(...)

In [17]:
from opsml_artifacts import ModelCard
from lightgbm import LGBMRegressor

model_registry = CardRegistry(registry_name="model") #load the model registry

data_splits = loaded_card.split_data() # get the data splits defined by split logic (data_card.data_splits)

# Prepare train data
data_splits.train.pop("EVAL_FLG") # pop off eval flg
y_train = data_splits.train.pop("DROP_OFF_TIME") # get train target

# Prepare test data
data_splits.test.pop("EVAL_FLG") # pop off eval flg
y_test = data_splits.test.pop("DROP_OFF_TIME") # get test target

# fit model
lgb_model = LGBMRegressor()
lgb_model.fit(data_splits.train, y_train)

model_card = ModelCard(
    trained_model=lgb_model,
    sample_input_data=data_splits.train[:1],
    name="tarp_lgb",
    team="SPMS", # defined above
    user_email="steven.forrester@shipt.com", # defined above
    data_card_uid=loaded_card.uid # this is required if you are planning on registering the model
)

{"message": "No connection args provided. Defaulting to local registry", "timestamp": "2023-02-09T01:04:01.839656Z", "app_env": "development", "level": "INFO"}


In [18]:
model_registry = CardRegistry(registry_name="model", connection_client=local_conn)
model_registry.register_card(card=model_card)

{"level": "INFO", "message": "Registering lightgbm onnx converter", "timestamp": "2023-02-09T01:04:18.498558Z", "app_env": "development", "host": null, "pid": 742575}
{"level": "INFO", "message": "Validating converted onnx model", "timestamp": "2023-02-09T01:04:18.808954Z", "app_env": "development", "host": null, "pid": 742575}
{"level": "INFO", "message": "Onnx model validated", "timestamp": "2023-02-09T01:04:18.841828Z", "app_env": "development", "host": null, "pid": 742575}
{"message": "MODEL_REGISTRY: tarp_lgb registered as version 1", "timestamp": "2023-02-09T01:04:18.874386Z", "app_env": "development", "level": "INFO"}


In [20]:
model_card = model_registry.load_card(uid=model_card.uid)
onnx_model = model_card.onnx_model()

In [21]:
onnx_model.input_sig.schema()

{'title': 'Features',
 'type': 'object',
 'properties': {'NBR_ADDRESSES': {'title': 'Nbr Addresses', 'type': 'integer'},
  'NBR_ORDERS': {'title': 'Nbr Orders', 'type': 'integer'},
  'NBR_RX': {'title': 'Nbr Rx', 'type': 'integer'},
  'NBR_APT': {'title': 'Nbr Apt', 'type': 'integer'},
  'METRO_X': {'title': 'Metro X', 'type': 'number'},
  'METRO_Y': {'title': 'Metro Y', 'type': 'number'},
  'METRO_Z': {'title': 'Metro Z', 'type': 'number'},
  'APT_FLG': {'title': 'Apt Flg', 'type': 'integer'}},
 'required': ['NBR_ADDRESSES',
  'NBR_ORDERS',
  'NBR_RX',
  'NBR_APT',
  'METRO_X',
  'METRO_Y',
  'METRO_Z',
  'APT_FLG']}

In [22]:
model_card.load_trained_model() # load trained model if laoding from registry

In [23]:
onnx_model.input_sig.schema()

{'title': 'Features',
 'type': 'object',
 'properties': {'NBR_ADDRESSES': {'title': 'Nbr Addresses', 'type': 'integer'},
  'NBR_ORDERS': {'title': 'Nbr Orders', 'type': 'integer'},
  'NBR_RX': {'title': 'Nbr Rx', 'type': 'integer'},
  'NBR_APT': {'title': 'Nbr Apt', 'type': 'integer'},
  'METRO_X': {'title': 'Metro X', 'type': 'number'},
  'METRO_Y': {'title': 'Metro Y', 'type': 'number'},
  'METRO_Z': {'title': 'Metro Z', 'type': 'number'},
  'APT_FLG': {'title': 'Apt Flg', 'type': 'integer'}},
 'required': ['NBR_ADDRESSES',
  'NBR_ORDERS',
  'NBR_RX',
  'NBR_APT',
  'METRO_X',
  'METRO_Y',
  'METRO_Z',
  'APT_FLG']}

In [25]:
import numpy as np
record = data_splits.test[0:1].to_dict(orient='records')[0]

# if testing a model that was trained on a numpy array, the model will expect a dictionary with a single list
# record = {"data": list(np.ravel(data[:1]))}

# test the onnx model 
onnx_pred = float(np.ravel(onnx_model.predict(record))[0])

# Compare to original model
orig_pred = float(onnx_model.predict_with_model(model_card.trained_model, record)[0])

print(f"Onnx: {round(onnx_pred,4)}", f"Lightgbm: {round(orig_pred,4)}")

Onnx: 7.1003 Lightgbm: 7.1003


In [26]:
model_registry.list_cards()

,uid,date,timestamp,app_env,name,team,version,user_email,model_card_uri,data_card_uid,trained_model_uri,sample_data_uri,sample_data_type,model_type
0,8e761efaaa2c45e19bb93a2d7a04a664,2023-02-07,1675792654344,development,test_model,mlops,3,mlops.com,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,aaf00ac219f6482386d61de10a8a89f2,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,ndarray,sklearn_estimator
1,677f061b24e44995858e788fb173fbc7,2023-02-07,1675806458392,development,test_model2,mlops,3,mlops.com,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,3eac180bd3704ed3944fcca25fb2ac56,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,ndarray,sklearn_estimator
2,43a5d38a52ce4b349f51f90462cd2d81,2023-02-07,1675788694904,development,test_model,mlops,2,mlops.com,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,c8160a98540240dca617d2db2fe56048,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,ndarray,sklearn_estimator
3,d2ef28de10774bbfa34ed16de9d1b9d2,2023-02-07,1675801823773,development,test_model2,mlops,2,mlops.com,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,0beb07f18da04d81b8d5545783f20a1b,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,ndarray,sklearn_estimator
4,7fdf2798e76e4be0b8b052653d913187,2023-02-07,1675806458392,development,test_model1,mlops,2,mlops.com,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,ec910d7fc2b744cc97cd14b37a31211e,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,ndarray,sklearn_estimator
5,e537b7ab05ec4f87aa6453d99a93c4bd,2023-02-07,1675788654445,development,test_model,mlops,1,mlops.com,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,56f8058f530a480e93b5823eb653b8a9,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,ndarray,sklearn_estimator
6,6afdc731e22d4e97a17ffe50ecf16082,2023-02-07,1675792654344,development,test_model2,mlops,1,mlops.com,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,aaf00ac219f6482386d61de10a8a89f2,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,ndarray,sklearn_estimator
7,e1b2710bce67440291e8e4485b489d93,2023-02-07,1675801823773,development,test_model1,mlops,1,mlops.com,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,45caf5b35edc49bfa14111be623172da,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,/home/steven.forrester/MODEL_REGISTRY/mlops/te...,ndarray,sklearn_estimator
8,3e8af6fcebc043819fe90e6bc95b240d,2023-02-09,1675904436562,development,tarp_lgb,SPMS,1,steven.forrester@shipt.com,/home/steven.forrester/MODEL_REGISTRY/SPMS/tar...,e18e5bd3fbb145a182d0fe61e24b1d66,/home/steven.forrester/MODEL_REGISTRY/SPMS/tar...,/home/steven.forrester/MODEL_REGISTRY/SPMS/tar...,DataFrame,lgbmregressor
